# 🎯 BNNR — Full Object Detection Demo on Pascal VOC 2007

**Bulletproof Neural Network Recipe** — Automated Augmentation Search for Detection.

This notebook demonstrates the **complete BNNR detection pipeline** on **Pascal VOC 2007** — a real-world detection benchmark with 20 object classes and thousands of images.

- Faster R-CNN with MobileNet backbone (ImageNet-pretrained)
- **All 9 BNNR detection augmentation types** — geometric, XAI-driven, composite
- **Bbox-aware** augmentations that automatically update bounding box coordinates
- **DetectionICD / DetectionAICD** — XAI-driven augmentation for detection (unique to BNNR)
- mAP@0.5 and mAP@[.5:.95] metrics with per-class AP
- Detection XAI: attention maps showing where the model focuses per object
- **Live dashboard** with detection-specific views

### What Makes BNNR Detection Special?

1. **Bbox-aware**: All augmentations that transform geometry (flip, rotate, scale) also update bounding box coordinates, clip them to image bounds, and remove degenerate boxes
2. **DetectionICD / AICD**: Uses bounding box regions as importance priors to create saliency-driven augmentations — unique to BNNR
3. **Mosaic & MixUp**: YOLOv4-style mosaic and alpha-blended mixup with proper box merging
4. **Automatic selection**: BNNR tests each augmentation candidate and keeps only those that improve mAP

### VOC 2007 Dataset

20 object classes: *aeroplane, bicycle, bird, boat, bottle, bus, car, cat, chair, cow, diningtable, dog, horse, motorbike, person, pottedplant, sheep, sofa, train, tvmonitor*

5,011 trainval + 4,952 test images — auto-downloads on first run (~460 MB, cached afterwards).

⏱ **Runtime:** ~25 min on Colab GPU (T4), ~90 min on CPU.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mateuszwalo/BNNR/blob/main/examples/detection/bnnr_detection_demo.ipynb)

## 1. Installation

In [ ]:
%pip install -q "bnnr[dashboard]" matplotlib

## 2. Imports

In [ ]:
import shutil
import tarfile
import urllib.request
from pathlib import Path

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision.datasets import VOCDetection

from bnnr import BNNRConfig, BNNRTrainer
from bnnr.detection_adapter import DetectionAdapter
from bnnr.detection_augmentations import (
    DetectionHorizontalFlip,
    DetectionVerticalFlip,
    DetectionRandomRotate90,
    DetectionRandomScale,
    MosaicAugmentation,
    DetectionMixUp,
)
from bnnr.detection_collate import detection_collate_fn_with_index
from bnnr.detection_icd import DetectionICD, DetectionAICD

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
print(f"Using device: {DEVICE}")

## 3. Dataset — Pascal VOC 2007

VOC 2007 contains **5,011 trainval** and **4,952 test** images across **20 object classes**:

*aeroplane, bicycle, bird, boat, bottle, bus, car, cat, chair, cow, diningtable, dog, horse, motorbike, person, pottedplant, sheep, sofa, train, tvmonitor*

Each image has bounding box annotations in XML format. Our `VOCDetectionDataset` wrapper:
- Resizes images to 300×300 (standard for Faster R-CNN)
- Rescales bounding boxes to match
- Returns BNNR's detection format: `(image_tensor, target_dict, index)`

### Data Contract
BNNR detection expects each sample to return a **3-tuple**:
```python
(image_tensor,  # [C, H, W] float32 ∈ [0, 1]
 target_dict,   # {"boxes": [N, 4] xyxy float32, "labels": [N] int64}
 index)          # int (sample index for caching)
```

**For your own data**, replace `VOCDetectionDataset` with your YOLO/COCO dataset — see Section 12 below.

In [ ]:
# ── VOC class definitions ────────────────────────────────────────────
VOC_CLASSES = [
    "aeroplane", "bicycle", "bird", "boat", "bottle",
    "bus", "car", "cat", "chair", "cow",
    "diningtable", "dog", "horse", "motorbike", "person",
    "pottedplant", "sheep", "sofa", "train", "tvmonitor",
]
VOC_CLASS_TO_IDX = {name: i + 1 for i, name in enumerate(VOC_CLASSES)}  # 0 = background
TARGET_SIZE = 300


# ── Robust VOC download — multiple mirrors with User-Agent ───────────
_VOC_MIRRORS = {
    "VOCdevkit_08-Jun-2007.tar": [
        "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCdevkit_08-Jun-2007.tar",
        "https://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCdevkit_08-Jun-2007.tar",
        "https://pjreddie.com/media/files/VOCdevkit_08-Jun-2007.tar",
    ],
    "VOCtrainval_06-Nov-2007.tar": [
        "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar",
        "https://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar",
        "https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar",
    ],
    "VOCtest_06-Nov-2007.tar": [
        "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar",
        "https://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar",
        "https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar",
    ],
}

def _download_voc_from_mirrors(root, image_set):
    """Download VOC 2007 archives from multiple mirrors."""
    root.mkdir(parents=True, exist_ok=True)
    filenames = ["VOCdevkit_08-Jun-2007.tar"]
    if image_set in ("trainval", "train", "val"):
        filenames.append("VOCtrainval_06-Nov-2007.tar")
    elif image_set == "test":
        filenames.append("VOCtest_06-Nov-2007.tar")
    for filename in filenames:
        target = root / filename
        if target.exists():
            try:
                with tarfile.open(target): pass
                with tarfile.open(target) as tar:
                    tar.extractall(path=root)
                print(f"  ✓ {filename} (cached)")
                continue
            except tarfile.TarError:
                target.unlink(missing_ok=True)
        mirrors = _VOC_MIRRORS.get(filename, [])
        ok = False
        for url in mirrors:
            try:
                print(f"  ↳ {url}")
                req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0 (BNNR)"})
                with urllib.request.urlopen(req, timeout=180) as r, target.open("wb") as out:
                    shutil.copyfileobj(r, out)
                with tarfile.open(target) as tar:
                    tar.extractall(path=root)
                print(f"  ✓ {filename}")
                ok = True
                break
            except Exception as exc:
                target.unlink(missing_ok=True)
                print(f"    ✗ {exc}")
        if not ok:
            raise RuntimeError(f"All mirrors failed for {filename}")


# ── VOC Dataset wrapper ───────────────────────────────────────────────
class VOCDetectionDataset(Dataset):
    """Wraps torchvision.VOCDetection → (image_tensor, target_dict, index).
    Tries 3 strategies: local data → torchvision download → custom mirrors.
    """

    def __init__(self, root="./data", image_set="trainval", target_size=300):
        self.root = Path(root)
        self.target_size = target_size

        # Strategy 1: data already exists locally
        try:
            self.voc = VOCDetection(root=str(self.root), year="2007",
                                    image_set=image_set, download=False)
            return
        except RuntimeError:
            pass
        # Strategy 2: torchvision built-in download
        try:
            print("  ↳ Trying torchvision download …")
            self.voc = VOCDetection(root=str(self.root), year="2007",
                                    image_set=image_set, download=True)
            return
        except Exception:
            pass
        # Strategy 3: custom mirrors with User-Agent header
        try:
            print("  ↳ Trying alternative mirrors …")
            _download_voc_from_mirrors(self.root, image_set)
            self.voc = VOCDetection(root=str(self.root), year="2007",
                                    image_set=image_set, download=False)
            return
        except Exception:
            pass
        raise RuntimeError(
            "VOC 2007 could not be loaded — all download mirrors are unavailable."
        )

    def __len__(self):
        return len(self.voc)

    def __getitem__(self, index):
        img, annotation = self.voc[index]
        orig_w, orig_h = img.size
        img_tensor = TF.resize(img, [self.target_size, self.target_size])
        img_tensor = TF.to_tensor(img_tensor)

        objs = annotation["annotation"]["object"]
        if not isinstance(objs, list):
            objs = [objs]

        boxes, labels = [], []
        for obj in objs:
            if obj.get("difficult", "0") == "1":
                continue
            name = obj["name"]
            if name not in VOC_CLASS_TO_IDX:
                continue
            bbox = obj["bndbox"]
            x1 = float(bbox["xmin"]) * self.target_size / orig_w
            y1 = float(bbox["ymin"]) * self.target_size / orig_h
            x2 = float(bbox["xmax"]) * self.target_size / orig_w
            y2 = float(bbox["ymax"]) * self.target_size / orig_h
            boxes.append([x1, y1, x2, y2])
            labels.append(VOC_CLASS_TO_IDX[name])

        if not boxes:
            boxes.append([0, 0, 1, 1])
            labels.append(0)

        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64),
        }
        return img_tensor, target, index


# ── Synthetic detection fallback (shapes with bboxes) ─────────────────
_SHAPE_COLORS = [
    (0.90, 0.22, 0.21), (0.30, 0.69, 0.31), (0.25, 0.32, 0.91),
    (0.98, 0.75, 0.18), (0.73, 0.33, 0.83),
]

class SyntheticDetectionDataset(Dataset):
    """Procedural detection dataset — zero downloads, works everywhere."""
    CLASS_NAMES = ["square", "circle", "triangle", "cross", "diamond"]

    def __init__(self, n_samples=500, target_size=300, seed=42):
        self.n_samples, self.target_size, self.seed = n_samples, target_size, seed

    def __len__(self):
        return self.n_samples

    def __getitem__(self, index):
        rng = random.Random(self.seed + index)
        S = self.target_size
        img = torch.zeros(3, S, S)
        br, bg, bb = rng.uniform(0.05, 0.2), rng.uniform(0.05, 0.2), rng.uniform(0.05, 0.2)
        ax = 0 if rng.random() > 0.5 else 1
        for ch, base in enumerate([br, bg, bb]):
            grad = torch.linspace(base, base + rng.uniform(0.05, 0.15), S)
            img[ch] = grad.unsqueeze(ax).expand(S, S)
        img += torch.randn_like(img) * 0.02
        img.clamp_(0.0, 1.0)
        boxes, labels = [], []
        for _ in range(rng.randint(1, 4)):
            ci = rng.randint(0, 4)
            w, h = rng.randint(S // 8, S // 3), rng.randint(S // 8, S // 3)
            x1, y1 = rng.randint(0, max(0, S - w - 1)), rng.randint(0, max(0, S - h - 1))
            x2, y2 = x1 + w, y1 + h
            cr, cg, cb = [c + rng.uniform(-0.08, 0.08) for c in _SHAPE_COLORS[ci]]
            if ci == 0:  # square
                img[0, y1:y2, x1:x2], img[1, y1:y2, x1:x2], img[2, y1:y2, x1:x2] = cr, cg, cb
            elif ci == 1:  # circle
                yy, xx = torch.meshgrid(torch.arange(S, dtype=torch.float32),
                                        torch.arange(S, dtype=torch.float32), indexing="ij")
                m = ((yy-(y1+y2)/2)/max(h/2,1))**2 + ((xx-(x1+x2)/2)/max(w/2,1))**2 <= 1
                img[0,m], img[1,m], img[2,m] = cr, cg, cb
            elif ci == 2:  # triangle
                for row in range(y1, y2):
                    frac = (row - y1) / max(y2 - y1, 1)
                    hw = int(frac * w / 2)
                    cx = (x1 + x2) // 2
                    img[:, row, max(0,cx-hw):min(S,cx+hw+1)] = torch.tensor([cr,cg,cb]).unsqueeze(1)
            elif ci == 3:  # cross
                aw, ah = max(2, w//5), max(2, h//5)
                cx, cy = (x1+x2)//2, (y1+y2)//2
                img[0,y1:y2,cx-aw:cx+aw], img[1,y1:y2,cx-aw:cx+aw], img[2,y1:y2,cx-aw:cx+aw] = cr,cg,cb
                img[0,cy-ah:cy+ah,x1:x2], img[1,cy-ah:cy+ah,x1:x2], img[2,cy-ah:cy+ah,x1:x2] = cr,cg,cb
            else:  # diamond
                yy, xx = torch.meshgrid(torch.arange(S, dtype=torch.float32),
                                        torch.arange(S, dtype=torch.float32), indexing="ij")
                m = torch.abs(xx-(x1+x2)/2)/max(w/2,1) + torch.abs(yy-(y1+y2)/2)/max(h/2,1) <= 1
                img[0,m], img[1,m], img[2,m] = cr, cg, cb
            boxes.append([float(x1), float(y1), float(x2), float(y2)])
            labels.append(ci + 1)
        target = {"boxes": torch.tensor(boxes, dtype=torch.float32),
                  "labels": torch.tensor(labels, dtype=torch.int64)}
        return img, target, index


# ── Load datasets — VOC with automatic synthetic fallback ─────────────
print("📦 Loading Pascal VOC 2007 (auto-downloads ~460 MB on first run)...")
use_synthetic = False
try:
    train_ds = VOCDetectionDataset(root="./data", image_set="trainval", target_size=TARGET_SIZE)
    val_ds = VOCDetectionDataset(root="./data", image_set="test", target_size=TARGET_SIZE)
    class_names = VOC_CLASSES
    NUM_CLASSES = len(VOC_CLASSES) + 1
except Exception as exc:
    print(f"\n⚠ VOC 2007 unavailable: {exc}")
    print("→ Using synthetic detection dataset (shapes with bboxes)\n")
    use_synthetic = True
    class_names = SyntheticDetectionDataset.CLASS_NAMES
    NUM_CLASSES = len(class_names) + 1
    train_ds = SyntheticDetectionDataset(n_samples=1000, target_size=TARGET_SIZE, seed=42)
    val_ds = SyntheticDetectionDataset(n_samples=300, target_size=TARGET_SIZE, seed=99)

# Use a subset for faster Colab demo (remove these 2 lines for full training)
train_ds = Subset(train_ds, range(min(1000, len(train_ds))))
val_ds = Subset(val_ds, range(min(500, len(val_ds))))

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True,
                          collate_fn=detection_collate_fn_with_index, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=4, shuffle=False,
                        collate_fn=detection_collate_fn_with_index, num_workers=2)

print(f"Train: {len(train_ds):,}  Val: {len(val_ds):,}  Classes: {len(class_names)}"
      f"  {'(synthetic fallback)' if use_synthetic else ''}")

### Visualize VOC samples

Real images with real bounding boxes.

In [ ]:
# 20 distinct colours for VOC classes
COLORS = [
    "#e74c3c", "#2ecc71", "#3498db", "#f39c12", "#9b59b6",
    "#1abc9c", "#e67e22", "#2980b9", "#c0392b", "#27ae60",
    "#8e44ad", "#d35400", "#16a085", "#f1c40f", "#e91e63",
    "#00bcd4", "#795548", "#607d8b", "#ff5722", "#4caf50",
]

fig, axes = plt.subplots(2, 4, figsize=(18, 9))
for i, ax in enumerate(axes.flat):
    img, target, _ = train_ds[i * 50]  # spread across dataset
    ax.imshow(img.permute(1, 2, 0).numpy())
    for box, label in zip(target["boxes"], target["labels"]):
        label_idx = int(label.item())
        if label_idx == 0:
            continue  # skip background
        x1, y1, x2, y2 = box.tolist()
        color = COLORS[(label_idx - 1) % len(COLORS)]
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1,
                                 linewidth=2, edgecolor=color, facecolor="none")
        ax.add_patch(rect)
        ax.text(x1, y1 - 3, class_names[label_idx - 1], fontsize=8,
                color="white", bbox=dict(facecolor=color, alpha=0.8, pad=1))
    ax.set_title(f"Sample {i * 50}", fontsize=10)
    ax.axis("off")
fig.suptitle(f"{'Pascal VOC 2007' if not use_synthetic else 'Synthetic Shapes'} — Detection Samples",
             fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

## 4. Model & Adapter

We use torchvision's **Faster R-CNN** with **MobileNet-v3 FPN** backbone, pre-trained on ImageNet for meaningful feature extraction.

`DetectionAdapter` wraps it for BNNR:
- **Train mode**: `model(images, targets)` returns loss dict
- **Eval mode**: `model(images)` returns prediction list
- **Metrics**: mAP@0.5 and mAP@[.5:.95] computed at epoch end

> **Note:** `num_classes` includes the background class, so for 20 VOC classes we pass `num_classes=21`.

In [ ]:
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn

model = fasterrcnn_mobilenet_v3_large_fpn(
    weights=None,
    weights_backbone="DEFAULT",   # ImageNet-pretrained backbone
    num_classes=NUM_CLASSES,       # 20 + 1 background = 21
)

optimizer = torch.optim.SGD(
    model.parameters(), lr=0.005, momentum=0.9, weight_decay=5e-4,
)

adapter = DetectionAdapter(
    model=model,
    optimizer=optimizer,
    device=DEVICE,
)

params = sum(p.numel() for p in model.parameters())
print(f"Faster R-CNN MobileNet-v3 FPN: {params:,} parameters")
print(f"Backbone: ImageNet-pretrained")

## 5. Detection Augmentations — All 9 BNNR Types

Detection augmentations are **bbox-aware** — they update bounding box coordinates when transforming images.

| Category | Augmentation | Effect | Bbox Update |
|:---|:---|:---|:---|
| **Geometric** | `DetectionHorizontalFlip` | Flip left-right | Mirrors box x-coordinates |
| **Geometric** | `DetectionVerticalFlip` | Flip top-bottom | Mirrors box y-coordinates |
| **Geometric** | `DetectionRandomRotate90` | Rotate by 90/180/270 | Rotates boxes accordingly |
| **Geometric** | `DetectionRandomScale` | Scale 0.85-1.15x | Scales box coordinates |
| **XAI-driven** | `DetectionICD` | Masks object tiles | Forces context learning |
| **XAI-driven** | `DetectionAICD` | Masks background tiles | Focuses on objects |
| **Composite** | `MosaicAugmentation` | 4-image mosaic | Merges boxes from all 4 |
| **Composite** | `DetectionMixUp` | Alpha-blend 2 images | Combines box targets |
| **Colour** | `AlbumentationsBboxAugmentation` | Brightness/contrast/hue | No box change (colour only) |

BNNR will test each candidate and **automatically keep only those that improve mAP**.

In [ ]:
augmentations = [
    # ---- Geometric (bbox-aware) ----
    DetectionHorizontalFlip(
        probability=0.5, name_override="det_hflip", random_state=SEED,
    ),
    DetectionVerticalFlip(
        probability=0.5, name_override="det_vflip", random_state=SEED + 1,
    ),
    DetectionRandomRotate90(
        probability=0.5, name_override="det_rotate90", random_state=SEED + 2,
    ),
    DetectionRandomScale(
        scale_range=(0.85, 1.15), probability=0.5,
        name_override="det_scale", random_state=SEED + 3,
    ),

    # ---- XAI-driven (unique to BNNR) ----
    DetectionICD(
        probability=0.5, threshold_percentile=70, tile_size=8,
        fill_strategy="gaussian_blur",
        name_override="det_icd", random_state=SEED + 10,
    ),
    DetectionAICD(
        probability=0.5, threshold_percentile=70, tile_size=8,
        fill_strategy="gaussian_blur",
        name_override="det_aicd", random_state=SEED + 11,
    ),

    # ---- Composite ----
    MosaicAugmentation(
        output_size=(TARGET_SIZE, TARGET_SIZE), probability=0.5,
        name_override="det_mosaic", random_state=SEED + 20,
    ),
    DetectionMixUp(
        alpha_range=(0.3, 0.7), probability=0.5,
        name_override="det_mixup", random_state=SEED + 21,
    ),
]

# ---- Albumentations colour jitter (if installed) ----
try:
    import albumentations as alb
    from bnnr.detection_augmentations import AlbumentationsBboxAugmentation

    augmentations.append(
        AlbumentationsBboxAugmentation(
            alb.Compose(
                [
                    alb.RandomBrightnessContrast(p=0.4),
                    alb.HueSaturationValue(p=0.3),
                ],
                bbox_params=alb.BboxParams(
                    format="pascal_voc",
                    label_fields=["labels"],
                    min_visibility=0.2,
                ),
            ),
            name_override="det_color_jitter",
            probability=0.5,
            random_state=SEED + 30,
        )
    )
except ImportError:
    print("(Albumentations not installed — skipping colour jitter candidate)")

print(f"{len(augmentations)} detection augmentation candidates:")
for i, aug in enumerate(augmentations, 1):
    tag = ""
    if "icd" in aug.name or "aicd" in aug.name:
        tag = " [XAI-driven]"
    elif "mosaic" in aug.name or "mixup" in aug.name:
        tag = " [composite]"
    elif "color" in aug.name or "jitter" in aug.name:
        tag = " [colour]"
    else:
        tag = " [geometric]"
    print(f"  {i:2d}. {aug.name:<24s}  p={aug.probability:.2f}{tag}")

### Visualize bbox-aware augmentations on real VOC images

See how geometric augmentations properly transform bounding boxes alongside images.

In [ ]:
# Pick a sample with clear objects
img_t, target, _ = train_ds[0]
img_np = (img_t.permute(1, 2, 0).numpy() * 255).astype(np.uint8)

# Apply 4 different augmentations for comparison
demo_augs = [
    ("Original", None),
    ("HFlip", DetectionHorizontalFlip(probability=1.0, random_state=1)),
    ("Rotate90", DetectionRandomRotate90(probability=1.0, random_state=1)),
    ("DetectionICD", DetectionICD(probability=1.0, threshold_percentile=70,
                                  tile_size=8, fill_strategy="gaussian_blur",
                                  random_state=1)),
]

fig, axes = plt.subplots(1, 4, figsize=(18, 5))
for ax, (title, aug) in zip(axes, demo_augs):
    if aug is None:
        im, tgt = img_np, target
    else:
        im, tgt = aug.apply_with_targets(
            img_np.copy(),
            {"boxes": target["boxes"].numpy(), "labels": target["labels"].numpy()},
        )

    ax.imshow(im)
    boxes = tgt["boxes"] if isinstance(tgt["boxes"], np.ndarray) else tgt["boxes"].numpy()
    labels = tgt["labels"] if isinstance(tgt["labels"], np.ndarray) else tgt["labels"].numpy()
    for box, label in zip(boxes, labels):
        label_idx = int(label)
        if label_idx == 0:
            continue
        x1, y1, x2, y2 = box
        color = COLORS[(label_idx - 1) % len(COLORS)]
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1,
                                 linewidth=2, edgecolor=color, facecolor="none")
        ax.add_patch(rect)
        ax.text(x1, y1 - 3, class_names[label_idx - 1], fontsize=8,
                color="white", bbox=dict(facecolor=color, alpha=0.8, pad=1))
    ax.set_title(title, fontsize=12)
    ax.axis("off")

fig.suptitle("Bbox-Aware Augmentations — Detection Samples",
             fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

## 6. Config

Setting `task="detection"` automatically configures:
- `selection_metric="map_50"` (optimize for mAP@0.5)
- `metrics=["map_50", "map_50_95", "loss"]`
- Detection-specific XAI (activation-based, fast)
- 512x512 preview and XAI images for clear visualization

In [ ]:
config = BNNRConfig(
    task="detection",
    m_epochs=3,               # Epochs per decision branch
    max_iterations=3,         # 3 rounds -> ~12 main-path epochs
    device=DEVICE,
    xai_enabled=True,
    detection_xai_method="activation",
    detection_xai_max_gt_boxes=3,
    detection_xai_max_pred_boxes=3,
    report_dir="reports/detection_voc_demo",
    report_preview_size=512,
    report_xai_size=512,
    save_checkpoints=False,     # Set True for production runs
    early_stopping_patience=2,
    detection_class_names=class_names,
    event_log_enabled=True,
    candidate_pruning_enabled=True,
    candidate_pruning_warmup_epochs=2,
    seed=SEED,
)

print(f"Task:             {config.task}")
print(f"Selection metric: {config.selection_metric}")
print(f"Metrics:          {config.metrics}")
print(f"XAI method:       {config.detection_xai_method}")
print(f"Preview size:     {config.report_preview_size}x{config.report_xai_size}")

## 7. Dashboard

Start dashboard before training so you can verify detection-specific signals in real time:
- branch tree (baseline vs candidates),
- `map_50` / `map_50_95` trends,
- sample previews with bbox overlays,
- detection XAI artifacts.

Tracking modes:
- **Local machine**: open local URL; terminal workflows also print QR for phone access.
- **Google Colab**: dashboard is embedded as an iframe.
- **Mobile from Colab (optional)**: use `pyngrok` with `NGROK_AUTHTOKEN`.

If dashboard cannot start, continue training and inspect `report.json` + `events.jsonl` after run.



In [ ]:
# Start dashboard server — runs in a background thread, doesn't block training.

import os
import time as _time

try:
    from bnnr import start_dashboard
    dashboard_url = start_dashboard(
        run_root=config.report_dir,
        port=8080,
        auto_open=False,  # We handle display below
        build_frontend=False,  # Skip npm build on Colab
    )
    print(f"Dashboard server: {dashboard_url}")

    # On Google Colab: embed dashboard in notebook
    try:
        import google.colab  # noqa: F401
        from google.colab import output as colab_output

        _time.sleep(2)
        print("\n📊 Colab detected — embedding live dashboard below:")
        colab_output.serve_kernel_port_as_iframe(8080, height=900)

        # Optional: expose dashboard externally (phone/outside Colab session)
        # Requires: NGROK_AUTHTOKEN set in environment
        try:
            from pyngrok import ngrok

            token = os.environ.get("NGROK_AUTHTOKEN")
            if token:
                ngrok.set_auth_token(token)
                public_url = ngrok.connect(8080, "http").public_url
                print(f"🌍 Public dashboard URL: {public_url}")
            else:
                print("ℹ Optional public URL: install pyngrok and set NGROK_AUTHTOKEN.")
        except Exception as tunnel_exc:
            print(f"ℹ Tunnel not started: {tunnel_exc}")

    except ImportError:
        # Not on Colab — open URL manually
        print("Open the URL above in your browser to view the dashboard.")

except Exception as e:
    print(f"Dashboard not available: {e}")
    print("Training will continue without live monitoring.")
    print("Results are saved to the reports/ folder.")


## 8. Train

This runs the full BNNR detection pipeline:
1. **Baseline** — train Faster R-CNN without augmentations to establish reference mAP
2. **Iteration 1-3** — test each augmentation candidate; keep those that improve mAP@0.5

At each step, BNNR generates:
- mAP@0.5, mAP@[.5:.95], and per-class AP metrics
- Detection XAI attention maps
- Augmentation preview images at 512x512

**Candidate pruning** is enabled: candidates that fall behind early are stopped, saving time.

In [ ]:
trainer = BNNRTrainer(adapter, train_loader, val_loader, augmentations, config)
result = trainer.run()

## 9. Results

In [ ]:
print("=" * 60)
print("  BNNR Detection Results — VOC 2007")
print("=" * 60)
print(f"  Best path  : {result.best_path}")
print(f"  Metrics    : {result.best_metrics}")
print(f"  Selected   : {result.selected_augmentations}")
print(f"  Report     : {result.report_json_path}")
print("=" * 60)

if result.selected_augmentations:
    print("\nBNNR selected these augmentations (in order of stacking):")
    for i, name in enumerate(result.selected_augmentations, 1):
        tag = ""
        if "icd" in name or "aicd" in name:
            tag = " <- XAI-driven!"
        elif "mosaic" in name or "mixup" in name:
            tag = " <- composite"
        print(f"  {i}. {name}{tag}")
    print("\nAll other candidates were tested but did not improve mAP.")

## 10. Inspect Detection XAI

Detection XAI images show where the model focuses attention for each detected object. Three panels: **Original with boxes | Attention heatmap | Overlay**.

In [ ]:
from IPython.display import display, Image as IPImage
import glob

run_dir = result.report_json_path.parent
xai_files = sorted(glob.glob(str(run_dir / "artifacts" / "xai" / "**" / "*.png"), recursive=True))

if xai_files:
    print(f"Found {len(xai_files)} detection XAI images. Showing first 6:")
    for f in xai_files[:6]:
        print(f"\n{Path(f).name}:")
        display(IPImage(filename=f, width=600))
else:
    print("No XAI images generated.")

# Also show augmentation previews
preview_files = sorted(glob.glob(str(run_dir / "artifacts" / "candidate_previews" / "*.png")))
if not preview_files:
    preview_files = sorted(glob.glob(str(run_dir / "artifacts" / "augmentation_previews" / "*.png")))
if preview_files:
    print(f"\nFound {len(preview_files)} augmentation preview images. Showing first 4:")
    for f in preview_files[:4]:
        print(f"\n{Path(f).name}:")
        display(IPImage(filename=f, width=600))

## 11. Explore the Events Log

Every training event is logged to `events.jsonl`. You can replay them to reconstruct the full training history.

In [ ]:
from bnnr import replay_events

events_path = run_dir / "events.jsonl"
if events_path.exists():
    events = replay_events(events_path)
    event_types = {}
    for e in events:
        t = e.get("event", "unknown")
        event_types[t] = event_types.get(t, 0) + 1

    print(f"Total events: {len(events)}")
    print("\nEvent types:")
    for t, count in sorted(event_types.items()):
        print(f"  {t:<35s} {count}")
else:
    print("No events.jsonl found.")

---

## 12. Using Your Own Data

### Option A: YOLO format (CLI)

```bash
bnnr train -c examples/configs/detection/detection_underwater.yaml \
    --dataset yolo \
    --data-path /path/to/your/data.yaml \
    --with-dashboard
```

### Option B: COCO format (CLI)

```bash
bnnr train -c examples/configs/detection/detection_underwater.yaml \
    --dataset coco_mini \
    --data-path /path/to/coco/dir \
    --with-dashboard
```

### Option C: Python API (full control)

Replace `VOCDetectionDataset` above with your own dataset class. The only requirement:

```python
def __getitem__(self, idx):
    return (
        image_tensor,  # [C, H, W] float32, values in [0, 1]
        {"boxes": boxes,  # [N, 4] float32, xyxy format
         "labels": labels},  # [N] int64
        idx,  # int
    )
```

See the [Custom Data Notebook](../bnnr_custom_data.ipynb) for detailed instructions.

---

## Next Steps

- **Full VOC 2007**: Remove the `Subset(...)` lines above and set `m_epochs=5`, `max_iterations=6`
- **Full showcase script**: Run `examples/detection/showcase_voc.py` for the complete experience
- **Custom YOLO data**: Use the CLI with `--dataset yolo --data-path your_data.yaml`
- **Classification**: See the [Classification Demo](../classification/bnnr_classification_demo.ipynb)
- **Augmentations**: See the [Augmentations Guide](../bnnr_augmentations_guide.ipynb)
- **Dashboard**: Run `bnnr dashboard serve --run-dir reports/detection_voc_demo`